<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Logistic Regression</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">


In [1]:
import config, src
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
data = src.load_data(config.PROCESSED_DATA_COMBINED_RAP_HIPHOP_FILE)

X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight = dict(zip(np.unique(y_train), class_weight))

In [ ]:
lr_classifier = LR(random_state=0, class_weight=class_weight, multi_class="multinomial")
lr_classifier.fit(X_train, y_train)

In [ ]:
src.evaluate_model(lr_classifier, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)

Voyons quelles variables sont les plus utiles à la prédiction:

In [ ]:
accuracy_variable = {}
for variables in X_train.columns:
    clf = LR(multi_class="multinomial",n_jobs=-1).fit(X_train[variables].to_numpy().reshape(-1, 1), y_train)
    y_pred = clf.predict(X_test[variables].to_numpy().reshape(-1, 1))
    accuracy_variable[variables] = accuracy_score(y_test, y_pred)

print(accuracy_variable)

Utilisons un algorithme de réduction de dimension pour augmenter le scoring:

In [ ]:
from sklearn.feature_selection import RFE

# Initialize a model and RFE
model = LR(multi_class="multinomial",n_jobs=-1).fit(X, y)
rfe = RFE(estimator=model, n_features_to_select=10)

# Fit RFE to the data
X_reduced = rfe.fit_transform(X, y)
print("Selected Features:", X.columns[rfe.support_])

Nouvelle regression après réduction de dimension, on a un gain de précision de 0.17:

In [ ]:
X_=X.loc[:,rfe.support_]

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, stratify=y)
clf = LR(multi_class="multinomial",n_jobs=-1).fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

Nous utilisons un gread search pour optimiser les paramètres "l1", "l2" et "elastic_net":

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'penalty': ['l1','l2', 'none'],
    'C': [0.01, 0.1, 1, 10, 100],
    'l1_ratio' : [0,0.01,1]
}


model = LR(multi_class="multinomial",n_jobs=-1)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)


On a donc une amélioration de 0.01 négligeable:

In [ ]:
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test Set Accuracy:", accuracy_score(y_test, y_pred))

---